In [1]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import json 
from pandas.io.json import json_normalize
import numpy as np



I have decided to use the BeautifulSoup library to get the postal codes,  Boroughs and Neighbours.
I have removed all rows, where Borough has value Not assigned.

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

url_content = requests.get(wiki_url).text
soup = BeautifulSoup(url_content, 'lxml')
bl = soup.find_all('table', {'class':'wikitable sortable'})


Borough = []
Neighbourhood = []
PostalCode = []


tab = bl[0].find_all('tr')    
for r in tab:
    cols = r.find_all('td')
    if len(cols) > 0:
        p = cols[0].text
        b = cols[1].text
        n = cols[2].text.rstrip()
        PostalCode.append(p)
        Borough.append(b)
        Neighbourhood.append(n)
            

df = pd.DataFrame()
nlist = {'PostalCode':PostalCode, 'Boroughs':Borough, 'Neighbours':Neighbourhood}
df = pd.DataFrame(nlist)
# Get rid with rows, where Boroughts are not assigned
df.drop(df[df.Boroughs == 'Not assigned'].index, inplace=True) 

I have replaced Neighbours with value Not assigned by value from column Boroughs

In [3]:
cond=df['Neighbours']=='Not assigned'
df['Neighbours']=df['Boroughs'].where(cond,df['Neighbours'])

I have combined rows with the same postal code into one row with the neighborhoods separated with a comma

In [4]:
df = df.sort_values(by='PostalCode', ascending=True)
df = df.groupby(['PostalCode','Boroughs'])['Neighbours'].apply(','.join).reset_index()

In [7]:
df.head()

,PostalCode,Boroughs,Neighbours
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df.shape

(103, 3)

-----------------------------------------